# IndicF5 Gradio Demo

This notebook requires a **GPU runtime**. Go to **Runtime → Change runtime type → T4 GPU** before running.

In [ ]:
# Check GPU availability - THIS MODEL REQUIRES GPU
import torch
if not torch.cuda.is_available():
    raise RuntimeError('❌ GPU not available! Go to Runtime → Change runtime type → T4 GPU')
print(f'✅ GPU available: {torch.cuda.get_device_name(0)}')

In [ ]:
# Clear HuggingFace cache to get fresh model code
!rm -rf ~/.cache/huggingface/modules/
print('✅ HuggingFace modules cache cleared')

In [ ]:
# Clone the repository (if not already present)
import os
if not os.path.exists('f15'):
    !git clone https://github.com/beginner4a3/f15
else:
    print('Repository already cloned.')

# Change working directory to the repo
os.chdir('f15')

In [ ]:
# Install compatible library versions
# Pin transformers<4.50 to avoid meta tensor issues
%pip install -q 'transformers<4.50' accelerate
%pip install -q git+https://github.com/ai4bharat/IndicF5.git
%pip install -q gradio spaces huggingface_hub
# Restart kernel after first run if you see import errors

In [ ]:
# Login to Hugging Face (optional for public models)
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Run the app
!python app.py